In [2]:
import pandas as pd
import os
import json
import pickle
import numpy as np

In [3]:
SAVE_DIR = "/home/aleflabo/ego_procedural/OadTR/assets/miniRoad"
ASSETS_DIR = "/home/aleflabo/ego_procedural/OadTR/assets"
DATAFOLDER = "/media/hdd/usr/edo/egoProcel_mistakes/data/mistake_labels"
ALL_CSV_NAMES = os.listdir(DATAFOLDER)
TRAIN_SPLIT = "/home/aleflabo/ego_procedural/OadTR/data/assembly/OadTR_assembly/train+val_cutGT_onlyThis/data_info_new.json"

In [4]:
j = json.load(open(TRAIN_SPLIT))
CORRECT_PROC = j["assembly"]["train_session_set"]
TOY2CLASS = "/media/hdd/usr/edo/egoProcel_mistakes/assets/toys.txt"

In [5]:
len(CORRECT_PROC), j["assembly"].keys()

(135, dict_keys(['class_index', 'train_session_set', 'test_session_set']))

In [6]:
IDXS_ACTIONS = "/home/aleflabo/ego_procedural/OadTR/data/assembly/OadTR_assembly/train+val_cutGT_onlyThis/mapping_noBG_trimmed.txt"
idxs_actions = np.loadtxt(IDXS_ACTIONS, dtype=str)
idx2action = {int(idx): action for idx, action in idxs_actions}
action2idx = {action: int(idx) for idx, action in idxs_actions}

In [13]:
# pickle.dump(idx2action, open(os.path.join(ASSETS_DIR, "idx2action.pkl"), "wb"))

In [7]:
toy2class_l = np.loadtxt(TOY2CLASS, dtype=str)[:, 1:]
toy2class = {t: c for t, c in toy2class_l}
toy2class

{'a01': 'excavator',
 'a02': 'bulldozer',
 'a03': 'bulldozer',
 'a06': 'clamp',
 'a07': 'bulldozer',
 'a08': 'crane',
 'a09': 'clamp',
 'a10': 'garbage_truck',
 'a11': 'bulldozer',
 'a12': 'dumper',
 'a13': 'dumper',
 'a14': 'transporter',
 'a15': 'garbage_truck',
 'a16': 'ladder_truck',
 'a17': 'fire_truck',
 'a18': 'fire_truck',
 'a19': 'ladder_truck',
 'a20': 'fire_truck',
 'a21': 'dumper',
 'a23': 'car',
 'a24': 'car',
 'a26': 'car',
 'a27': 'suv',
 'a28': 'suv',
 'a29': 'suv',
 'a30': 'suv',
 'a31': 'car',
 'b01a': 'bulldozer',
 'b01b': 'roller',
 'b02a': 'dumper',
 'b02b': 'clamp',
 'b03a': 'ladder_truck',
 'b03b': 'fire_truck',
 'b04a': 'excavator',
 'b04b': 'bulldozer',
 'b04c': 'jackhammer',
 'b04d': 'roller',
 'b05a': 'excavator',
 'b05b': 'crane',
 'b05c': 'cement_mixer',
 'b05d': 'dumper',
 'b06a': 'dumper',
 'b06b': 'crane',
 'b06c': 'excavator',
 'b06d': 'cement_mixer',
 'b08a': 'clamp',
 'b08b': 'bulldozer',
 'b08c': 'bulldozer',
 'b08d': 'dumper',
 'c01a': 'excavator',


In [7]:
# json.dump(toy2class, open(os.path.join(ASSETS_DIR, "toy2class.json"), "w"))

In [8]:
idx2emoji = json.load(open(os.path.join(ASSETS_DIR, "idx2emoji.json")))
idx2emoji

{'0': {'utf-8': '\\u2744\\ufe0f', 'escape': '❄️'},
 '1': {'utf-8': '\\U0001f3da\\ufe0f', 'escape': '🏚️'},
 '2': {'utf-8': '\\U0001f6cc\\U0001f3fb', 'escape': '🛌🏻'},
 '3': {'utf-8': '\\U0001f693', 'escape': '🚓'},
 '4': {'utf-8': '\\U0001f998', 'escape': '🦘'},
 '5': {'utf-8': '\\U0001f6d5', 'escape': '🛕'},
 '6': {'utf-8': '\\U0001f4e1', 'escape': '📡'},
 '7': {'utf-8': '\\U0001faa4', 'escape': '🪤'},
 '8': {'utf-8': '\\U0001f332', 'escape': '🌲'},
 '9': {'utf-8': '\\U0001f90c\\U0001f3fd', 'escape': '🤌🏽'},
 '10': {'utf-8': '\\U0001f378', 'escape': '🍸'},
 '11': {'utf-8': '\\U0001f31a', 'escape': '🌚'},
 '12': {'utf-8': '\\U0001f9ce\\U0001f3fc', 'escape': '🧎🏼'},
 '13': {'utf-8': '\\U0001f349', 'escape': '🍉'},
 '14': {'utf-8': '\\U0001f90f\\U0001f3ff', 'escape': '🤏🏿'},
 '15': {'utf-8': '\\U0001f450\\U0001f3fb', 'escape': '👐🏻'},
 '16': {'utf-8': '\\U0001f446\\U0001f3fc', 'escape': '👆🏼'},
 '17': {'utf-8': '\\U0001f61a', 'escape': '😚'},
 '18': {'utf-8': '\\U0001f68c', 'escape': '🚌'},
 '19': {'utf-8

In [9]:
out = {}
for csv_name in CORRECT_PROC:
    csv_path = os.path.join(DATAFOLDER, f"{csv_name}.csv")
    df = pd.read_csv(csv_path)
    toy = csv_name.split("-")[2].split("_")[0]
    toy_class = toy2class[toy]

    s = df["verb"] + "-" + df["this"]
    s_alpha = s.apply(lambda x: x.replace(" ", "_")).values.tolist()
    s_num = [action2idx[alpha] for alpha in s_alpha]
    s_emoji = [idx2emoji[str(idx)]["escape"] for idx in s_num]

    if toy_class not in out:
        out[toy_class] = {csv_name: {"alpha": s_alpha, "num": s_num, "emoji": s_emoji}}
    else:
        out[toy_class][csv_name] = {"alpha": s_alpha, "num": s_num, "emoji": s_emoji}

In [10]:
out = {}
for csv_name in CORRECT_PROC:
    csv_path = os.path.join(DATAFOLDER, f"{csv_name}.csv")
    df = pd.read_csv(csv_path)
    toy = csv_name.split("-")[2].split("_")[0]
    # toy_class = toy2class[toy]

    s = df["verb"] + "-" + df["this"]
    s_alpha = s.apply(lambda x: x.replace(" ", "_")).values.tolist()
    s_num = [action2idx[alpha] for alpha in s_alpha]
    s_emoji = [idx2emoji[str(idx)]["escape"] for idx in s_num]

    if toy not in out:
        out[toy] = {csv_name: {"alpha": s_alpha, "num": s_num, "emoji": s_emoji}}
    else:
        out[toy][csv_name] = {"alpha": s_alpha, "num": s_num, "emoji": s_emoji}
out

{'a21': {'nusar-2021_action_both_9081-a21_9081_user_id_2021-02-12_155024': {'alpha': ['attach-bumper',
    'attach-cabin',
    'attach-base',
    'attach-dump_bed',
    'attach-wheel'],
   'num': [39, 74, 58, 13, 37],
   'emoji': ['🦵🏿', '🃏', '🫓', '🍉', '🕜']},
  'nusar-2021_action_both_9032-a21_9032_user_id_2021-02-04_112105': {'alpha': ['attach-chassis',
    'attach-base',
    'attach-cabin',
    'attach-base',
    'attach-wheel',
    'attach-bumper'],
   'num': [32, 58, 74, 58, 37, 39],
   'emoji': ['♑', '🫓', '🃏', '🫓', '🕜', '🦵🏿']},
  'nusar-2021_action_both_9062-a21_9062_user_id_2021-02-09_151231': {'alpha': ['attach-cabin',
    'attach-base',
    'attach-dump_bed',
    'attach-wheel'],
   'num': [74, 58, 13, 37],
   'emoji': ['🃏', '🫓', '🍉', '🕜']}},
 'c03d': {'nusar-2021_action_both_9021-c03d_9021_user_id_2021-02-23_100036': {'alpha': ['attach-cabin',
    'attach-grill',
    'attach-chassis',
    'attach-body',
    'attach-dump_bed',
    'attach-wheel'],
   'num': [74, 80, 32, 61, 13, 

In [12]:
json.dump(out, open(os.path.join(SAVE_DIR, "assembly_context_seqs_train_onlyToy.json"), "w"))

In [13]:
def get_prompt(toy, seq):
    hist = [-1] + seq[:-1]
    next_sym = seq[-1]
    prompt = f"Sequence type: {toy}\n"
    prompt += f"Input Sequence:\n {', '.join(map(str,hist))}\n"
    prompt += f"Next Symbol:\n {next_sym}\n---\n"
    return prompt

In [14]:
out_context = {}
for toy_class, v1 in out.items():
    prompt_num, prompt_alpha, prompt_emoji = "", "", ""
    for seq, v2 in v1.items():
        toy = seq.split("-")[2].split("_")[0]
        prompt_num += get_prompt(toy, v2["num"])
        prompt_alpha += get_prompt(toy, v2["alpha"])
        prompt_emoji += get_prompt(toy, v2["emoji"])
    out_context[toy_class] = {
        "num": prompt_num,
        "alpha": prompt_alpha,
        "emoji": prompt_emoji,
    }

In [15]:
out_context #["dumper"]

{'a21': {'num': 'Sequence type: a21\nInput Sequence:\n -1, 39, 74, 58, 13\nNext Symbol:\n 37\n---\nSequence type: a21\nInput Sequence:\n -1, 32, 58, 74, 58, 37\nNext Symbol:\n 39\n---\nSequence type: a21\nInput Sequence:\n -1, 74, 58, 13\nNext Symbol:\n 37\n---\n',
  'alpha': 'Sequence type: a21\nInput Sequence:\n -1, attach-bumper, attach-cabin, attach-base, attach-dump_bed\nNext Symbol:\n attach-wheel\n---\nSequence type: a21\nInput Sequence:\n -1, attach-chassis, attach-base, attach-cabin, attach-base, attach-wheel\nNext Symbol:\n attach-bumper\n---\nSequence type: a21\nInput Sequence:\n -1, attach-cabin, attach-base, attach-dump_bed\nNext Symbol:\n attach-wheel\n---\n',
  'emoji': 'Sequence type: a21\nInput Sequence:\n -1, 🦵🏿, 🃏, 🫓, 🍉\nNext Symbol:\n 🕜\n---\nSequence type: a21\nInput Sequence:\n -1, ♑, 🫓, 🃏, 🫓, 🕜\nNext Symbol:\n 🦵🏿\n---\nSequence type: a21\nInput Sequence:\n -1, 🃏, 🫓, 🍉\nNext Symbol:\n 🕜\n---\n'},
 'c03d': {'num': 'Sequence type: c03d\nInput Sequence:\n -1, 74, 80,

In [16]:
json.dump(
    out_context,
    open(os.path.join(SAVE_DIR, "assembly_context_prompt_train_onlyToy.json"), "w"),
)

In [19]:
j = json.load(open(TRAIN_SPLIT))
MISTAKE_PROC = j["assembly"]["test_session_set"]


for csv_name in MISTAKE_PROC:
    print(csv_name.split("-")[2].split("_")[0])

a08
b08d
c12e
a14
a24
c02b
a23
c13d
c12c
a20
a30
c01c
c03e
a07
c04c
b04a
c13d
c12d
c12a
c05a
c08c
a31
b05b
c03a
a16
a14
c04a
c05a
a31
c12b
b05b
c11a
a27
b06c
c04b
c09c
c03e
c14b
c01d
b05a
c02c
c10b
c02a
a06
b08b
c07b
a27
b05a
c08c
c01c
c08c
a06
b05d
a08
a13
c14b
c12e
a06
c02a
c12c
a17
c09b
b08a
c07b
a02
c06f
c06c
a10
b04c
b08c
c06e
c13a
b05a
b05c
a10
a09
a30
c08b
a09
c09b
c04c
b04a
c02a
b05c
c04b
b08b
c11a
c06d
a28
c11b
c01b
c10b
a17
a08
b04d
c10b
b06a
b01a
b04d
c12d
c04d
c13e
c10b
c08b
c10c
c11a
c07a
a13
b04d
c13a
c13b
b06c
c09c
c04b
a03
c10a
c13a
c09a
c12b
a14
c02a
a31
c13a
c06b
b05b
c01c
c02c
c08a
a17
b01a
c06e
c05b
a19
b01b
c13a
c08c
c13a
a09
a16
c12d
c07c
c08a
c06a
c04d
a11
b08b
a14
b02b
a24
c14a
a01
c13b
c07a
c03a
b01a
c12e
b02b
b06a
a27
c14a
c09c
c06a
a28
c13a
c03e
b03b
c12c
c12a
c04a
b04c
b01b
c09a
a23
b01b
a03
c13f
b03a
c07b
c07b
c09c
a31
c01d
